In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

In [10]:
def encoder(df, col_split):
    continuous_cols = df.columns[col_split:]
    categorical_cols = df.columns[:col_split]

    # Standardizing the continuous features
    scaler = StandardScaler()
    df[continuous_cols] = scaler.fit_transform(df[continuous_cols])

    # Applying One-Hot Encoding to categorical features
    encoder = ColumnTransformer(transformers=[('cat', OneHotEncoder(sparse_output=True), categorical_cols)], remainder='passthrough')
    return encoder.fit_transform(df)


In [11]:
def load_data():
    df = pd.read_csv('beta_dates/beta_data_7_60.csv', index_col=0)
    y = pd.read_csv('beta_dates/true_labels.csv', index_col=0).values[:,0]
    y = y + 1
    X = encoder(df, 4)
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)

    return X_train, X_test, y_train, y_test


# XGBoost Grid Search

In [31]:
X_train, X_test, y_train, y_test = load_data()
# create a pipeline for XGBoost, SVD and GridSearchCV
pipe = Pipeline([
        ('svd', TruncatedSVD()),
        ('xgb', xgb.XGBClassifier(objective='multi:softmax', random_state=42)),
        ])

# create a parameter grid for the pipeline

param_grid = {
        'svd__n_components': [3,5],
        'xgb__n_estimators': [80, 110],  # Reduced number of trees
        'xgb__learning_rate': [0.01, 0.1, 1],  # Expanded range with a lower bound
        'xgb__max_depth': [3,4],  # Shallower trees considering fewer components
        'xgb__min_child_weight': [2, 4],  # Adjusted values for instance weight
        'xgb__gamma': [0.3, 0.5],  # Slightly expanded range for loss reduction
        'xgb__subsample': [0.5, 0.8],  # Adjusted subsample ratio
        'xgb__colsample_bytree': [0.8, 1],  # Adjusted subsample ratio of columns
        'xgb__reg_alpha': [1,3],  # Adjusted L1 regularization term
        'xgb__reg_lambda': [2,3],  # Adjusted L2 regularization term
        }

# perform grid search

grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# train model with best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Predictions
predictions = best_model.predict(X_test)

# Evaluation
print(classification_report(y_test, predictions))

print(grid_search.best_params_)
print(grid_search.best_score_)

print(grid_search.cv_results_)

Fitting 3 folds for each of 1536 candidates, totalling 4608 fits
              precision    recall  f1-score   support

         0.0       0.50      0.25      0.33         8
         1.0       0.83      0.80      0.81        54
         2.0       0.31      0.57      0.40         7

    accuracy                           0.71        69
   macro avg       0.54      0.54      0.51        69
weighted avg       0.74      0.71      0.71        69

{'svd__n_components': 3, 'xgb__colsample_bytree': 1, 'xgb__gamma': 0.3, 'xgb__learning_rate': 1, 'xgb__max_depth': 4, 'xgb__min_child_weight': 2, 'xgb__n_estimators': 110, 'xgb__reg_alpha': 3, 'xgb__reg_lambda': 3, 'xgb__subsample': 0.8}
0.7854504120488776
{'mean_fit_time': array([0.01676695, 0.01604732, 0.01456571, ..., 0.01806768, 0.01958394,
       0.01933503]), 'std_fit_time': array([0.00065626, 0.00079285, 0.00180926, ..., 0.0042155 , 0.00059534,
       0.00401533]), 'mean_score_time': array([0.00106041, 0.00077756, 0.00067568, ..., 0.00060193

# XGBoost Random Search

In [24]:
X_train, X_test, y_train, y_test = load_data()
# create a pipeline for XGBoost, SVD, and RandomizedSearchCV
pipe = Pipeline([
        ('svd', TruncatedSVD()),
        ('xgb', xgb.XGBClassifier(objective='multi:softmax', random_state=42)),
        ])

# create a parameter grid for the pipeline
param_grid = {
        'svd__n_components': [3,4,5],
        'xgb__gamma': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4, 200],
        'xgb__learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.300000012, 0.4, 0.5, 0.6, 0.7],
        'xgb__max_depth': [5,6,7,8,9,10,11,12,13,14],
        'xgb__n_estimators': [50,65,80,100,115,130,150],
        'xgb__reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
        'xgb__reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
        'xgb__subsample': [0.5,0.6,0.7,0.8,0.9,1],
        'xgb__colsample_bytree': [0.5,0.6,0.7,0.8,0.9,1],
        'xgb__min_child_weight': [0.5,1,3,5,7,9],
              }

# perform random search

random_search = RandomizedSearchCV(estimator=pipe, param_distributions=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1, n_iter=1000)
random_search.fit(X_train, y_train)

# train model with best parameters
best_model = random_search.best_estimator_
best_model.fit(X_train, y_train)

# Predictions
predictions = best_model.predict(X_test)

# Evaluation
print(classification_report(y_test, predictions))

print(random_search.best_params_)
print(random_search.best_score_)
print(random_search.cv_results_)

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits
              precision    recall  f1-score   support

         0.0       0.25      0.12      0.17         8
         1.0       0.74      0.89      0.81        47
         2.0       0.75      0.43      0.55        14

    accuracy                           0.71        69
   macro avg       0.58      0.48      0.51        69
weighted avg       0.68      0.71      0.68        69

{'xgb__subsample': 0.6, 'xgb__reg_lambda': 0.4, 'xgb__reg_alpha': 0, 'xgb__n_estimators': 50, 'xgb__min_child_weight': 3, 'xgb__max_depth': 8, 'xgb__learning_rate': 0.300000012, 'xgb__gamma': 0.8, 'xgb__colsample_bytree': 0.5, 'svd__n_components': 5}
0.7904944586530265
{'mean_fit_time': array([0.01006818, 0.0228014 , 0.01074028, 0.01033274, 0.01736236,
       0.01348162, 0.01605439, 0.00964769, 0.00995731, 0.00937192,
       0.00870148, 0.01336567, 0.01582678, 0.01216706, 0.02707386,
       0.01923537, 0.02848355, 0.01631133, 0.01353574, 0.03025866

# XGBoost Bayesian Optimization

In [39]:
X_train, X_test, y_train, y_test = load_data()

# Create a pipeline
pipe = Pipeline([
    ('svd', TruncatedSVD(n_components=5)),
    ('xgb', xgb.XGBClassifier(objective='multi:softmax', random_state=42)),
])

# Define the evaluation function
def xgb_evaluate(gamma, learning_rate, max_depth, n_estimators, reg_alpha, reg_lambda, subsample, colsample_bytree, min_child_weight):
    params = {
        'xgb__gamma': gamma,
        'xgb__learning_rate': learning_rate,
        'xgb__max_depth': int(max_depth),
        'xgb__n_estimators': int(n_estimators),
        'xgb__reg_alpha': reg_alpha,
        'xgb__reg_lambda': reg_lambda,
        'xgb__subsample': subsample,
        'xgb__colsample_bytree': colsample_bytree,
        'xgb__min_child_weight': min_child_weight,
    }
    pipe.set_params(**params)
    pipe.fit(X_train, y_train)
    predictions = pipe.predict(X_test)
    return accuracy_score(y_test, predictions)

# Bayesian Optimization
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds={
        'gamma': (0, 5),
        'learning_rate': (0.01, 0.5),
        'max_depth': (3, 5),
        'n_estimators': (50, 300),
        'reg_alpha': (1, 100),
        'reg_lambda': (1, 100),
        'subsample': (0.5, 1.0),
        'colsample_bytree': (0.5, 1.0),
        'min_child_weight': (1, 10),
    },
    random_state=42
)

optimizer.maximize(init_points=5, n_iter=200)

# After optimization, retrieve and adjust best parameters for pipeline
best_params = optimizer.max['params']
best_params_for_pipeline = {'xgb__' + key: int(value) if key in ['max_depth', 'n_estimators'] else value for key, value in best_params.items()}

# Set the best parameters in the pipeline
pipe.set_params(**best_params_for_pipeline)
pipe.fit(X_train, y_train)

# Predictions and Evaluation
predictions = pipe.predict(X_test)
print(classification_report(y_test, predictions))

# Print best parameters and performance
print("Best Parameters for Pipeline:", best_params_for_pipeline)
print("Best Performance:", optimizer.max['target'])



|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.7536    | 0.6873    | 4.754     | 0.3687    | 4.197     | 2.404     | 89.0      | 6.75      | 86.75     | 0.8006    |
| 2         | 0.7536    | 0.854     | 0.1029    | 0.4853    | 4.665     | 2.911     | 95.46     | 19.16     | 31.12     | 0.7624    |
| 3         | 0.7536    | 0.716     | 1.456     | 0.3098    | 3.279     | 3.629     | 141.6     | 46.15     | 78.73     | 0.5998    |
| 4         | 0.1014    | 0.7571    | 2.962     | 0.03276   | 4.215     | 2.535     | 66.26     | 94.94     | 96.6      | 0.9042    |
| 5         | 0.7536    | 0.6523    | 0.4884    | 0.3453    | 3.88      | 2.098     | 173.8     | 4.404     | 91.02     | 0.6294    |
| 6         | 0.7536    | 0.5       | 5.0       | 0.01      | 

# Build the best model

In [12]:
X_train, X_test, y_train, y_test = load_data()
# create a pipeline for XGBoost, FAMD, Random Forests, and Multiclass logistic regression
pipe = Pipeline([
        ('svd', TruncatedSVD()),
        ('xgb', xgb.XGBClassifier(objective='multi:softmax', random_state=42)),
        ])

# create a parameter grid for the pipeline

param_grid = {
'svd__n_components': 5,
 'xgb__colsample_bytree': 0.8, 
 'xgb__gamma': 0.5, 
 'xgb__learning_rate': 0.1,
   'xgb__max_depth': 3, 
   'xgb__min_child_weight': 2, 
   'xgb__n_estimators': 200, 
   'xgb__reg_alpha': 1, 
   'xgb__reg_lambda': 2, 
   'xgb__subsample': 0.7}


pipe.fit(X_train, y_train).score(X_test, y_test)

pipe.set_params(**param_grid).fit(X_train, y_train).score(X_test, y_test)

# Predictions
predictions = pipe.predict(X_test)

# Evaluation
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.80      0.44      0.57         9
         1.0       0.83      0.88      0.85        50
         2.0       0.55      0.60      0.57        10

    accuracy                           0.78        69
   macro avg       0.73      0.64      0.67        69
weighted avg       0.78      0.78      0.78        69



# Random Forest Grid search

In [18]:
X_train, X_test, y_train, y_test = load_data()
pipe = Pipeline([
        ('svd', TruncatedSVD()),
        ('rf', RandomForestClassifier()),
])

param_grid = {
        'svd__n_components': [3,4,5],
        'rf__n_estimators': [50, 100, 200],  # Number of trees in the forest
        'rf__max_depth': [10, 20, 30],  # Maximum depth of each tree
        'rf__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
        'rf__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required at each leaf node
        'rf__max_features': ['sqrt', 'log2'],  # Number of features to consider when looking for the best split
}

# perform grid search

grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# train model with best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Predictions
predictions = best_model.predict(X_test)

# Evaluation
print(classification_report(y_test, predictions))

print(grid_search.best_params_)
print(grid_search.best_score_)

print(grid_search.cv_results_)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits
              precision    recall  f1-score   support

         0.0       0.25      0.50      0.33         2
         1.0       0.82      0.87      0.85        54
         2.0       0.50      0.31      0.38        13

    accuracy                           0.75        69
   macro avg       0.52      0.56      0.52        69
weighted avg       0.75      0.75      0.74        69

{'rf__max_depth': 10, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 50, 'svd__n_components': 4}
0.7755754475703324
{'mean_fit_time': array([0.03956795, 0.04751801, 0.03917408, 0.0917093 , 0.10956375,
       0.1008064 , 0.14807359, 0.15691932, 0.1661423 , 0.05882533,
       0.04342675, 0.03912862, 0.06663585, 0.07155434, 0.07748206,
       0.14668433, 0.15844329, 0.17339635, 0.03562593, 0.03707639,
       0.04201571, 0.07453386, 0.09103815, 0.07146708, 0.15757847,
       0.14986269, 0.15080206,

# Random Forest Bayesian Search

In [44]:
X_train, X_test, y_train, y_test = load_data()

# Create a pipeline for Random Forests
pipe = Pipeline([
    ('svd', TruncatedSVD(n_components=5)),
    ('rf', RandomForestClassifier(random_state=42)),
])

# Define the evaluation function
def rf_evaluate(n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features):
    params = {
        'rf__n_estimators': int(n_estimators),
        'rf__max_depth': int(max_depth),
        'rf__min_samples_split': int(min_samples_split),
        'rf__min_samples_leaf': int(min_samples_leaf),
        'rf__max_features': max_features,
    }
    pipe.set_params(**params)
    pipe.fit(X_train, y_train)
    predictions = pipe.predict(X_test)
    return accuracy_score(y_test, predictions)

# Bayesian Optimization
optimizer = BayesianOptimization(
    f=rf_evaluate,
    pbounds={
        'n_estimators': (50, 300),
        'max_depth': (3, 5),
        'min_samples_split': (2, 10),
        'min_samples_leaf': (1, 4),
        'max_features': (0.1,1)
    },
    random_state=42
)

optimizer.maximize(init_points=5, n_iter=200)

# After optimization, retrieve and adjust best parameters for pipeline
best_params = optimizer.max['params']
best_params_for_pipeline = {'rf__' + key: int(value) if key in ['max_depth', 'n_estimators', 'min_samples_split', 'min_samples_leaf'] else value for key, value in best_params.items()}

# Set the best parameters in the pipeline
pipe.set_params(**best_params_for_pipeline)
pipe.fit(X_train, y_train)

# Predictions and Evaluation
predictions = pipe.predict(X_test)
print(classification_report(y_test, predictions))

# Print best parameters and performance
print("Best Parameters for Pipeline:", best_params_for_pipeline)
print("Best Performance:", optimizer.max['target'])



|   iter    |  target   | max_depth | max_fe... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------
| 1         | 0.7971    | 3.749     | 0.9556    | 3.196     | 6.789     | 89.0      |
| 2         | 0.7681    | 3.312     | 0.1523    | 3.599     | 6.809     | 227.0     |
| 3         | 0.8116    | 3.041     | 0.9729    | 3.497     | 3.699     | 95.46     |
| 4         | 0.7826    | 3.367     | 0.3738    | 2.574     | 5.456     | 122.8     |
| 5         | 0.7536    | 4.224     | 0.2255    | 1.876     | 4.931     | 164.0     |
| 6         | 0.7826    | 3.0       | 0.1       | 1.0       | 10.0      | 103.8     |
| 7         | 0.8261    | 4.825     | 0.6208    | 3.332     | 2.16      | 100.4     |
| 8         | 0.7971    | 5.0       | 1.0       | 4.0       | 2.0       | 107.2     |
| 9         | 0.8116    | 5.0       | 0.1       | 2.281     | 2.0       | 74.66     |
| 10        | 0.8116    | 4.915     | 0.5955    | 3.77

# Logistic Regression Grid Search

In [17]:
pipe = Pipeline([
        ('svd', TruncatedSVD()),
        ('logreg', LogisticRegression()),
])

param_grid = {
        'svd__n_components': [3,4,5],
        'logreg__C': [0.1, 1, 10, 100],
        'logreg__penalty': ['l1', 'l2'],
        'logreg__multi_class': ['ovr', 'multinomial'],
        'logreg__solver': ['newton-cg', 'lbfgs', 'sag', 'saga']
}

# perform grid search

grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# train model with best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Predictions
predictions = best_model.predict(X_test)

# Evaluation
print(classification_report(y_test, predictions))

print(grid_search.best_params_)
print(grid_search.best_score_)

print(grid_search.cv_results_)


Fitting 3 folds for each of 192 candidates, totalling 576 fits


/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter 

              precision    recall  f1-score   support

         0.0       1.00      0.25      0.40         8
         1.0       0.68      0.91      0.78        43
         2.0       0.60      0.33      0.43        18

    accuracy                           0.68        69
   macro avg       0.76      0.50      0.54        69
weighted avg       0.70      0.68      0.64        69

{'logreg__C': 10, 'logreg__multi_class': 'multinomial', 'logreg__penalty': 'l1', 'logreg__solver': 'saga', 'svd__n_components': 5}
0.7704603580562659
{'mean_fit_time': array([0.00138402, 0.00125567, 0.00140532, 0.00119885, 0.00123   ,
       0.00119297, 0.00097648, 0.00090496, 0.00098578, 0.00425299,
       0.00408729, 0.00398207, 0.00584769, 0.00514436, 0.00724037,
       0.00289194, 0.00271336, 0.00271535, 0.00431999, 0.00277034,
       0.00312122, 0.00556231, 0.00468095, 0.00346398, 0.00060034,
       0.0010097 , 0.00067735, 0.00060829, 0.00102957, 0.00073902,
       0.00066996, 0.00061591, 0.00066201, 0.0025

/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter 

# Bayesian Optimization

In [52]:
# Assume load_data function and dataset are defined
X_train, X_test, y_train, y_test = load_data()

# Create a pipeline for Logistic Regression
pipe = Pipeline([
    ('svd', TruncatedSVD(n_components=5)),
    ('logreg', LogisticRegression(max_iter=1000)),  # Increased max_iter for convergence
])

penalties = ['l2']
multi_classes = ['ovr', 'multinomial']
solvers = ['newton-cg', 'lbfgs', 'sag', 'saga']

# Define the evaluation function
def logreg_evaluate(C, penalty, multi_class, solver):
    penalty = penalties[int(penalty)]
    multi_class = multi_classes[int(multi_class)]
    solver = solvers[int(solver)]

    params = {
        'logreg__C': C,
        'logreg__penalty': penalty,
        'logreg__multi_class': multi_class,
        'logreg__solver': solver,
    }
    pipe.set_params(**params)
    pipe.fit(X_train, y_train)
    predictions = pipe.predict(X_test)
    return accuracy_score(y_test, predictions)

# Bayesian Optimization
optimizer = BayesianOptimization(
    f=logreg_evaluate,
    pbounds={
        'C': (0.1, 100),
        'penalty': (0, len(penalties) - 1),  # Ensuring valid index range
        'multi_class': (0, len(multi_classes) - 1),  # Ensuring valid index range
        'solver': (0, len(solvers) - 1),  # Ensuring valid index range
    },
    random_state=42
)

optimizer.maximize(init_points=5, n_iter=200)

# After optimization, retrieve and adjust best parameters for pipeline
best_params = optimizer.max['params']
best_params_for_pipeline = {
    'logreg__C': best_params['C'],
    'logreg__penalty': penalties[int(best_params['penalty'])],
    'logreg__multi_class': multi_classes[int(best_params['multi_class'])],
    'logreg__solver': solvers[int(best_params['solver'])],
}

# Set the best parameters in the pipeline
pipe.set_params(**best_params_for_pipeline)
pipe.fit(X_train, y_train)

# Predictions and Evaluation
predictions = pipe.predict(X_test)
print(classification_report(y_test, predictions))

# Print best parameters and performance
print("Best Parameters for Pipeline:", best_params_for_pipeline)
print("Best Performance:", optimizer.max['target'])


|   iter    |  target   |     C     | multi_... |  penalty  |  solver   |
-------------------------------------------------------------------------
| 1         | 0.7391    | 37.52     | 0.9507    | 0.0       | 1.796     |
| 2         | 0.7391    | 15.69     | 0.156     | 0.0       | 2.599     |
| 3         | 0.7391    | 60.15     | 0.7081    | 0.0       | 2.91      |
| 4         | 0.7391    | 83.26     | 0.2123    | 0.0       | 0.5502    |
| 5         | 0.7391    | 30.49     | 0.5248    | 0.0       | 0.8737    |
| 6         | 0.7391    | 99.98     | 0.1644    | 0.0       | 2.847     |
| 7         | 0.7391    | 5.017     | 0.01848   | 0.0       | 2.171     |
| 8         | 0.6957    | 0.5474    | 0.04037   | 0.0       | 1.056     |
| 9         | 0.7391    | 97.08     | 0.7096    | 0.0       | 1.122     |
| 10        | 0.7391    | 98.91     | 0.4378    | 0.0       | 1.358     |
| 11        | 0.7391    | 9.113     | 0.05824   | 0.0       | 2.922     |
| 12        | 0.7391    | 89.17     | 

# Gaussian Discriminant Analysis Grid Search

In [12]:
X_train, X_test, y_train, y_test = load_data()

pipe = Pipeline([
        ('svd', TruncatedSVD()),
        ('gdareg', GaussianNB()),
])

param_grid = {
        'svd__n_components': [3,4,5],
        'gdareg__priors': [[0.05,0.9,0.05],[0.1,0.8,0.1],[0.15,0.7,0.15],[0.2,0.6,0.2]]
}

# perform grid search

grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# train model with best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Predictions
predictions = best_model.predict(X_test)

# Evaluation
print(classification_report(y_test, predictions))

print(grid_search.best_params_)
print(grid_search.best_score_)

print(grid_search.cv_results_)


Fitting 3 folds for each of 12 candidates, totalling 36 fits
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         9
         1.0       0.65      1.00      0.79        44
         2.0       1.00      0.06      0.12        16

    accuracy                           0.65        69
   macro avg       0.55      0.35      0.30        69
weighted avg       0.64      0.65      0.53        69

{'gdareg__priors': [0.05, 0.9, 0.05], 'svd__n_components': 5}
0.721938050582552
{'mean_fit_time': array([0.09349036, 0.05310734, 0.02721842, 0.00428843, 0.00660189,
       0.00937184, 0.00425776, 0.00373745, 0.00528232, 0.0040532 ,
       0.00427826, 0.00580279]), 'std_fit_time': array([0.049119  , 0.06027522, 0.02243783, 0.00119649, 0.00539769,
       0.00622216, 0.00102889, 0.00050375, 0.00178973, 0.0005825 ,
       0.00070931, 0.00313598]), 'mean_score_time': array([0.0048945 , 0.00523694, 0.00170978, 0.00167855, 0.00120552,
       0.00213345, 0.001

/home/xanderdelabru/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xanderdelabru/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xanderdelabru/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Discriminant Analysis Bayesian Optimization

In [16]:
# Assume load_data function and dataset are defined
X_train, X_test, y_train, y_test = load_data()

# Create a pipeline for Logistic Regression
pipe = Pipeline([
    ('svd', TruncatedSVD(n_components=5)),
    ('gdareg', GaussianNB()),  # Increased max_iter for convergence
])

priors = [[0.05,0.9,0.05],[0.1,0.8,0.1],[0.15,0.7,0.15],[0.2,0.6,0.2]]

# Define the evaluation function
def gdareg_evaluate(prior):
    prior = priors[int(prior)]

    params = {
        'gdareg__priors': prior
    }
    pipe.set_params(**params)
    pipe.fit(X_train, y_train)
    predictions = pipe.predict(X_test)
    return accuracy_score(y_test, predictions)

# Bayesian Optimization
optimizer = BayesianOptimization(
    f=gdareg_evaluate,
    pbounds={
        'prior': (0, len(priors) - 1),  # Ensuring valid index range
    },
    random_state=42
)

optimizer.maximize(init_points=5, n_iter=200)

# After optimization, retrieve and adjust best parameters for pipeline
best_params = optimizer.max['params']
best_params_for_pipeline = {
    'gdareg__priors': priors[int(best_params['prior'])],
}

# Set the best parameters in the pipeline
pipe.set_params(**best_params_for_pipeline)
pipe.fit(X_train, y_train)

# Predictions and Evaluation
predictions = pipe.predict(X_test)
print(classification_report(y_test, predictions))

# Print best parameters and performance
print("Best Parameters for Pipeline:", best_params_for_pipeline)
print("Best Performance:", optimizer.max['target'])


|   iter    |  target   |   prior   |
-------------------------------------
| 1         | 0.7101    | 1.124     |
| 2         | 0.6522    | 2.852     |
| 3         | 0.6522    | 2.196     |
| 4         | 0.7101    | 1.796     |
| 5         | 0.7826    | 0.4681    |
| 6         | 0.7101    | 1.124     |
| 7         | 0.7826    | 0.09951   |
| 8         | 0.7826    | 0.284     |
| 9         | 0.7826    | 1.661e-05 |
| 10        | 0.7826    | 0.6284    |
| 11        | 0.7826    | 0.5579    |
| 12        | 0.7826    | 0.3703    |
| 13        | 0.7826    | 0.1879    |
| 14        | 0.7826    | 0.04225   |
| 15        | 0.7826    | 0.6878    |
| 16        | 0.7826    | 0.4191    |
| 17        | 0.7826    | 0.2358    |
| 18        | 0.7826    | 0.662     |
| 19        | 0.7826    | 0.5139    |
| 20        | 0.7826    | 0.1433    |
| 21        | 0.7826    | 0.3271    |
| 22        | 0.7826    | 0.5923    |
| 23        | 0.7826    | 0.07106   |
| 24        | 0.7826    | 0.7166    |
| 25        